# Modul Daten

Dieses Jupyter Notebook soll einen Überblick über die Aufbereitung, Bereinigung und Visualisierung von Daten vermitteln. 

Für diesen Zweck wurde die Datenbank Gas Sensor Array Drift verwendet, heruntergeladen und vorher für diesen Zweck manipuliert (Gas- und Konzentrationsklassenlaben wurden zu einem Label zusammen geführt und dabei drei verschiedene Konzentrationsstufen definiert: low, medium und high). <br>
Der vollständige Datensatz kann auf <br>

http://archive.ics.uci.edu/ml/datasets/Gas+Sensor+Array+Drift+Dataset+at+Different+Concentrations

heruntergeladen werden. Außerdem werden verschiedene Informationen zu den Daten bereitgestellt. <br>

Der Datensatz enthält 13.910 Messungen von 16 chemischen Sensoren, welche abwechselnd sechs verschiedene Gase (Ethanol, Ethen, Ammoniak, Acetaldehyd, Aceton und Toluol) in verschiedenen Konzentrationen untersuchen. <br>
Der Datensatz wurde zwischen Januar 2008 und Februar 2011 (36 Monaten) erhoben. <br>
Jede Messung liefert anhand der 16 vorhandenen Sensoren eine 16-Kanal-Zeitreihe. <br>
Es werden zwei Hauptfeatures in diesem Datensatz betrachtet: <br>
$\rightarrow~$ (i) das stationäre Feature (steady-state) bezeichnet als `DR`, definiert als die maximale Widerstandsänderung in Bezug auf eine Basis, sowie die normalisierte Version davon (`|DR|`). <br>
$\rightarrow~$ (ii) ein Ansammlung an Features welche die Sensordynamik der gesamten Messung wiederspiegelt (`EMAi` und `EMAd` für verschiedene $\alpha$-Werte). <br>

Im folgendem Beispiel werden wir uns nur mit dem `DR` Feature auseinandersetzen. Da jeder der 16 Sensoren diesen Messwert liefert, sind auch 16 verschiedene Messwerte von `DR` vorhanden (`DR_1` des ersten Sensors, `DR_2`, des zweiten Sensorns und wo weiter). 

## Laden des Pandas DataFrames

Als erstes müssen alle Beispieldatensätze geladen werden. Dieser liegen in 10 verschiedene csv Dateien vor (batch0.csv bis batch9.csv). Anschließend werden diese zu einem gemeinsamen Datensatz zusammengeführt.

In [2]:
import scripts.load

scripts.load.load()

(                 target           DR_1     |DR|_1  EMAi0.001_1  EMAi0.01_1  \
 0           Ethanol low     509.641237   2.033514     0.254809    2.167413   
 1           Ethanol low   38533.391702   2.342134     4.010622    9.777162   
 2           Ethanol low   30640.213902   3.371802     9.387105    8.130167   
 3           Ethanol low   26495.225267   3.473369     8.957996   14.171439   
 4           Ethanol low   56839.500510   4.289495     8.446959   15.191184   
 ...                 ...            ...        ...          ...         ...   
 13905        Aceton low   23266.320775   3.093495     2.044893    6.236694   
 13906  Acetaldehyd high   30808.306384   2.879310    11.029082   14.524925   
 13907       Aceton high  186222.896547  15.721410    44.446638   57.718414   
 13908  Acetaldehyd high   24907.601420   2.062939     5.162683    7.165311   
 13909       Aceton high  234281.659124  19.232700    61.822330   88.741905   
 
         EMAi0.1_1  EMAd0.001_1  EMAd0.01_1   EMAd

In [3]:
df_modified, df_original,all_files_mod,all_files_original = scripts.load.load()


## Ein kurzer Blick auf die Datenstruktur

Um ein Gefühl für die Datenstruktur zu erhalten bietet Pandas die folgenden Methoden an: <br>

`head(n)` $\longrightarrow~$ Gibt die $n$ ersten Zeilen (default $n$ = 5) aus. Nützlich um schnell zu testen, ob das Objekt im richtigen Datentyp vorliegt. <br>
`info()` $\longrightarrow~$ Gibt eine übersichtliche Zusammenfassung des DataFrames aus, einschließlich des Index dtypes, des Spalten dtypes, Nicht-Null-Werte sowie den Speicherverbrauch. <br>
`describe()` $\longrightarrow~$ Erstellt deskriptive Statistiken, welche die zentrale Tendenz, Verteilung und Form der Verteilung eines Datensatzes unter Ausschluss von NaN-Werten zusammenfasst. <br>
`value_count()` $\longrightarrow~$ Gibt die Anzahl eindeutiger Werte in absteigender Reihenfolge wieder. Muss an einem Feature aufgerufen werden.

Werfen wir als erstes ein Blick auf die `head()` Methode:

In [4]:
df_modified.head()

,target,DR_1,|DR|_1,EMAi0.001_1,EMAi0.01_1,EMAi0.1_1,EMAd0.001_1,EMAd0.01_1,EMAd0.1_1,DR_2,...,EMAd0.01_15,EMAd0.1_15,DR_16,|DR|_16,EMAi0.001_16,EMAi0.01_16,EMAi0.1_16,EMAd0.001_16,EMAd0.01_16,EMAd0.1_16
0,Ethanol low,509.641237,2.033514,0.254809,2.167413,6.917387,-1.071371,-0.326185,-1.357455,29716.668137,...,-1.018086,-2.932405,1376.997851,4.436690,0.450471,1.537286,0.174129,-0.668878,-1.533990,-1.869676
1,Ethanol low,38533.391702,2.342134,4.010622,9.777162,7.463293,-3.601266,-3.688345,-1.030023,7076.796078,...,-1.021407,-2.730868,4309.679899,4.338311,1.055154,1.718463,2.943279,-0.696066,-1.026806,-2.789014
2,Ethanol low,30640.213902,3.371802,9.387105,8.130167,18.610025,-4.571710,-17.002459,-8.314741,28209.204322,...,-2.875316,-2.044677,4859.314837,4.812519,1.411884,2.794603,2.218621,-1.126425,-1.681372,-1.677701
3,Ethanol low,26495.225267,3.473369,8.957996,14.171439,32.514786,-12.360699,-2.027190,-2.039976,30649.650834,...,-2.959944,-3.223345,6955.286998,5.567068,1.841280,3.546894,3.323281,-0.744751,-2.685686,-1.864186
4,Ethanol low,56839.500510,4.289495,8.446959,15.191184,16.894776,-12.490107,-4.025442,-2.401690,66156.665575,...,-2.999126,-5.519291,7639.317741,6.308976,1.605233,3.132163,4.705466,-2.159960,-3.049733,-3.343077


Die erste Spalte verät uns das Klassenlabel. Die erste Instanz ist demnach eine Messung am Gas Ethanol bei niedriger Konzentration. Alle weitere Spalten beinhalten Daten die zu diesem Zeitpunkt bzw. Messpunkt erhoben worden sind. <br>

Rufen wir die `info()` Methode an den ersten fünf Spalten auf.

In [5]:
df_modified.iloc[:,:5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13910 entries, 0 to 13909
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   target       13910 non-null  object 
 1   DR_1         13892 non-null  float64
 2   |DR|_1       13891 non-null  float64
 3   EMAi0.001_1  13896 non-null  float64
 4   EMAi0.01_1   13889 non-null  float64
dtypes: float64(4), object(1)
memory usage: 543.5+ KB


Für Pandas Indexing und Slicing siehe https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html. <br>

`info()` listet uns die verschiedene dtypes auf: vier float64 und eins vom Typ object. Anhand der unterschiedlichen Instanzen (total: 13910, DR_1: 13892, ...) erkennen wir, dass der Datensatz unvollständig ist (NaNs können ein Grund dafür sein). <br>

Werfen wir als nächstes ein Blick auf die `describe()` Methode.

In [6]:
df_modified.iloc[:,:5].describe()

,DR_1,|DR|_1,EMAi0.001_1,EMAi0.01_1
count,13892.000000,13891.000000,13896.000000,13889.000000
mean,50414.936606,6.643495,12.938526,18.715159
std,70124.851629,13.581118,17.692590,24.998014
min,-32269.296506,-7.661152,-11.945900,-14.244902
25%,6878.411415,2.279255,1.732025,2.553648
50%,20749.781769,3.997641,5.314358,7.685242
75%,62852.469633,8.631351,17.177674,25.949883
max,700870.254384,1342.294514,168.362526,231.740498


`describe()` bietet zusätzliche Informationen zu den jeweiligen Features. Dazu gehört die Anzahl, der Mittelwert (mean), die Standardabweichung (std) sowie das minimale, 25%, 50%, 75% und das maximale Quantil.

In [7]:
df_modified['target'].value_counts()

Aceton high           1834
Toluol low            1425
Ethen low             1190
Ammoniak high         1156
Ethanol low           1083
Ethen high            1082
Ethanol high           963
Acetaldehyd high       836
Ethen medium           654
Aceton low             642
Acetaldehyd low        565
Acetaldehyd medium     535
Aceton medium          533
Ethanol medium         519
Toluol medium          397
Ammoniak medium        311
Ammoniak low           174
Toluol high             11
Name: target, dtype: int64

Wir haben `value_counts()` an der `target` Spalte aufgerufen und sehen das Aceton medium mit 1924 Einträge über die meisten Messungen verfügt. Gefolgt von Toluol low und so weiter.

### Pandas Profiling

Im folgenden soll noch Pandas Profiling vorgestellt werden. Pandas Profiling bietet ähnlich wie die `describe()` Methode statistische Auswertungen. Mehr dazu kann auf https://github.com/pandas-profiling/pandas-profiling gelesen werden. <br>

Der folgende Code (auskommentiiert da dieser Schritt zeitintensiv ist) erstellt ein vollständigen Bericht und speichert diesen als `report.html` im lokalen Verzeichnis ab.

In [8]:
#profile = df_modified.profile_report(title='Pandas Profiling Report')
#profile.to_file(output_file="report.html")